In [3]:
rom keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
# create the model
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128)
from keras.models import model_from_json
import numpy
from keras.preprocessing import sequence
from keras import backend as K


def run(text_input):  
    K.clear_session()
	f = open("model_architecture.json",'r+')
	json_string = f.read()
	f.close()
	model = model_from_json(json_string)

	model.load_weights('final_lstm.h5')
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

	max_words = 500

	word_index = imdb.get_word_index()
	
	s = numpy.array([word_index.get(word, 0) for word in text_input.split()])

	s = sequence.pad_sequences([s], maxlen=max_words)

	y_pred = model.predict_classes(s)

	if y_pred[0] == 1:
		return "Positive"
	else:
		return "Negative"
	K.clear_session()
from flask import Flask, render_template, flash, redirect, url_for, session, request, logging
from wtforms import Form, BooleanField, StringField, validators
import os
import sys
import imdb_classifer

app = Flask(__name__)
app.debug=True
app.secret_key = "super secret key"

@app.route('/')
def index():
    return render_template('home.html')

@app.route('/home')
def home():
    return render_template('home.html')


class RegisterForm(Form):
    name = StringField('', [validators.Length(min=1, max=500)], render_kw={"placeholder": "Please Enter a statement here"})



# connect and run the python backend
@app.route('/code', methods=['GET', 'POST'])
def register():
    form = RegisterForm(request.form)
    if request.method == 'POST' and form.validate():
        name = form.name.data
        

        # call the run function from the imported python file
        out = imdb_classifer.run(name)
        output=str(out)

        return render_template("outputs/"+output+'.html')
    return render_template('code.html', form=form)


if __name__=='__main__':
    app.run()
    
    
    
    

SyntaxError: invalid syntax (<ipython-input-3-2a2a950091b2>, line 1)